### Softmax Classifier for CIFAT-10

#### Load_Dataset : Constomize CIFAR-10 Image Data 

In [3]:
import numpy as np
import tensorflow as tf
import time
import Load_Dataset
from PIL import Image

#### load Dataset from Load_Dataset.py

In [4]:
data_set = Load_Dataset.load_data()
data_reshape = Load_Dataset.reshape_data(data_set)

train_X = data_reshape['images_train']
train_Y = data_reshape['labels_train']
test_X = data_reshape['images_test']
test_Y = data_reshape['labels_test']

In [11]:
X = tf.placeholder(shape=[None, 32, 32, 3], dtype=tf.float32, name='X')
Y = tf.placeholder(shape=[None], dtype=tf.uint8, name='Y')
W = tf.Variable(tf.zeros([3072, 10]), name='W')
B = tf.Variable(tf.zeros([10]), name='B')

In [18]:
def layer(input, weight_shape, name):
    with tf.name_scope(name):
        W = tf.Variable(tf.truncated_normal(shape=weight_shape, mean=0.0, stddev=0.02, name='layer_W'))
        L = tf.nn.conv2d(input, W, strides=[1,1,1,1], padding='SAME', name='conv')
        relu = tf.nn.relu(L, name='relu')
        output = tf.nn.max_pool(L, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name='pooling')
        return W, output

In [32]:
def fully_connect(input, weight_shape, bias_shape, name):
    with tf.name_scope(name):
        W = tf.get_variable('ful_W', shape=weight_shape, initializer=tf.contrib.layers.xavier_initializer())
        B = tf.Variable(tf.zeros(bias_shape), name='ful_B')
        H = tf.matmul(input, W, name='ful_H') + B
        softmax = tf.nn.softmax(H, name='softmaxw')
        return H, softmax

In [33]:
## Layer 1 => 0-W, 1-B, 2-W, 3-output
layer_1 = layer(X, [2, 2, 3, 32], 'first')[1]

layer_2 = layer(layer_1, [2, 2, 32, 64], 'second')[1]

# reshape Last output to vector
vec_layer = tf.reshape(layer_2, [-1, 8*8*64])

In [34]:
## fully connect => 0-output, 1-sofrmax
ful_output, softmax = fully_connect(vec_layer, [8*8*64, 10], [10], 'ful1')
print(ful_output)
print(softmax)

Tensor("ful1_3/add:0", shape=(?, 10), dtype=float32)
Tensor("ful1_3/softmaxw:0", shape=(?, 10), dtype=float32)


In [8]:
## one_hot label
Y_one = tf.one_hot(Y, 10, axis=1)

In [9]:
## minimize
entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y_one, logits=ful_output, name='loss')
loss = tf.reduce_mean(entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(loss)

In [10]:
# Accuracy
Accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(softmax, 1), tf.argmax(Y_one, 1)), dtype=tf.float32))

In [14]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    epoch = 100
    batch_size = 10000
    batch_count = 50000 // batch_size
    for i in range(epoch):
        print("epoch :", i)
        loss_total=0
        for i in range(batch_count):
            batch_index = i*batch_size
            img = np.reshape(train_X[batch_index:batch_index+batch_size], [-1,3072])
            label = train_Y[batch_index:batch_index+batch_size]
            _,loss_val = sess.run([train_op, loss],feed_dict={X:img, Y:label})
            loss_total += loss_val
        print("loss ==> ", loss_total)

    test_img = np.reshape(test_X[:], [-1, 3072])
    test_label = test_Y[:]
    acc = sess.run([Accuracy], feed_dict={X:test_img, Y:test_label})
    print("Accuracy : ", acc) 

epoch : 0
loss ==>  137.075204372
epoch : 1
loss ==>  24.0965044498
epoch : 2
loss ==>  16.718909502
epoch : 3
loss ==>  12.5249843597
epoch : 4
loss ==>  11.0780494213
epoch : 5
loss ==>  9.90970230103
epoch : 6
loss ==>  9.47206318378
epoch : 7
loss ==>  9.17908418179
epoch : 8
loss ==>  8.93826425076
epoch : 9
loss ==>  8.76218008995
epoch : 10
loss ==>  8.63074219227
epoch : 11
loss ==>  8.48962903023
epoch : 12
loss ==>  8.37132656574
epoch : 13
loss ==>  8.2615377903
epoch : 14
loss ==>  8.16776096821
epoch : 15
loss ==>  8.07616198063
epoch : 16
loss ==>  7.98842215538
epoch : 17
loss ==>  7.90094685555
epoch : 18
loss ==>  7.81561112404
epoch : 19
loss ==>  7.73667562008
epoch : 20
loss ==>  7.66360008717
epoch : 21
loss ==>  7.59478294849
epoch : 22
loss ==>  7.52839446068
epoch : 23
loss ==>  7.45912826061
epoch : 24
loss ==>  7.39351928234
epoch : 25
loss ==>  7.32578790188
epoch : 26
loss ==>  7.2594666481
epoch : 27
loss ==>  7.19594204426
epoch : 28
loss ==>  7.1251713037